#Amazon Review- Tagging Negative Review in Amazon Product Review with CNN Model

#Introduction


### Connecting to Golab Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importing Libraries

In [2]:
import functools
import sys
import csv
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import tqdm
import time


### Dataset

In [9]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
vectors = torchtext.vocab.FastText()

.vector_cache/wiki.en.vec: 6.60GB [00:54, 122MB/s]                            
100%|██████████| 2519370/2519370 [06:04<00:00, 6909.34it/s]


In [61]:
df_train = '/content/drive/MyDrive/Masters_Thesis/Dataset/topicReviewText/topicReviewText_train.csv'
df_test = '/content/drive/MyDrive/Masters_Thesis/Dataset/topicReviewText/orgtopicReviewText_test.csv'
target_list = 'encoded_sub_topic'
train_dataloader, valid_dataloader, vocab_size, pad_index, output_dim, vocab = get_data_loaders(train_dataframe= df_train,valid_dataframe=df_test,target_list=target_list)

In [62]:
df_train = '/content/drive/MyDrive/Masters_Thesis/Dataset/encoded_data/encoded_sub_topic_train.csv'
df_test = '/content/drive/MyDrive/Masters_Thesis/Dataset/encoded_data/encoded_sub_topic_test.csv'

train_dataloader_encoded, valid_dataloader_encoded, vocab_size, pad_index, output_dim, vocab = get_data_loaders(train_dataframe= df_train,valid_dataframe=df_test,target_list=target_list)

In [39]:
for _,data in enumerate(train_dataloader, 0):
      # for batch in tqdm.tqdm(dataloader, desc='training...', file=sys.stdout):
        print(data['ids'])
        print(data['length'])
        print(data[target_list])
        print(data)
        break

tensor([[526,  86, 142,  ...,   1,   1,   1],
        [ 27,  45,  82,  ...,   1,   1,   1],
        [390,   2, 371,  ...,   1,   1,   1],
        ...,
        [ 23, 109, 103,  ...,   1,   1,   1],
        [817,  46,   8,  ...,   1,   1,   1],
        [  3,  85,  67,  ...,   1,   1,   1]])
tensor([ 51,  24,  15,  93,   9,  53,  49,   5,  17, 100,  72,   3,  21,  19,
         69,  28,  46,  49,   9, 100,  80,  20,  33,  26,   8,  24,  23,  14,
          8,  43,  29,   9,  64,  10,  31,  95,  43,  20, 256,   3,  58,  33,
         29, 217,  61,  82, 120,   3,  40,  27,  41,  33,  88,  28,  16,  32,
         40, 102, 157, 256, 132,  20,  37, 110])
tensor([4, 2, 1, 3, 3, 2, 0, 2, 4, 1, 2, 3, 2, 4, 3, 1, 4, 4, 0, 1, 4, 3, 3, 3,
        3, 2, 1, 4, 2, 2, 0, 4, 1, 1, 3, 4, 1, 2, 4, 2, 2, 4, 4, 0, 4, 4, 4, 2,
        3, 2, 3, 2, 4, 2, 1, 4, 4, 0, 1, 4, 3, 4, 1, 2])
{'ids': tensor([[526,  86, 142,  ...,   1,   1,   1],
        [ 27,  45,  82,  ...,   1,   1,   1],
        [390,   2, 371,  ...,   

In [40]:
for _,data in enumerate(valid_dataloader, 0):
      # for batch in tqdm.tqdm(dataloader, desc='training...', file=sys.stdout):
        print(data['ids'])
        print(data['length'])
        print(data[target_list])
        print(data)
        break

tensor([[  18,  111,   19,  ...,    1,    1,    1],
        [  75,   27,   20,  ...,    1,    1,    1],
        [1462,    5,   34,  ...,    1,    1,    1],
        ...,
        [ 719,   14,  235,  ...,    1,    1,    1],
        [   9,   12,    8,  ...,    1,    1,    1],
        [  18,  143,   12,  ...,    1,    1,    1]])
tensor([ 32,  50,  12,  67,  13,  37, 107,   7,  12,  44,  31,   4, 144,  40,
         39,  68,  41,  14,  32, 256,  34, 132, 172,  36,  98, 145, 181,  35,
        232, 110, 109, 107,  27,   6,  22,  93,  56,  20,  58, 134, 190,  31,
         18,  31,  16,  13,  15, 256,  57, 102,  24, 193,  18,  94,  24, 105,
         68,  55,  63,  36,  24,   5,  11, 181])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
{'ids': tensor([[  18,  111,   19,  ...,    1,    1,    1],
        [  75,   27,   20,  ...,    

### Model Implementation

In [68]:
embedding_dim = 300
hidden_dim = 300
n_layers = 2
bidirectional = True
dropout_rate = 0.5

model = CNN(vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout_rate, 
             pad_index)
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,793,705 trainable parameters


In [69]:
model = apply_weights(model,vocab)

lr = 5e-4

optimizer = optim.Adam(model.parameters(), lr=lr)
loss_function = nn.CrossEntropyLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
loss_function = loss_function.to(device)


### CNN Train & Valid with reviewText and encoded_sub_topiic

In [66]:
train(total_epoch=20, model=model, train_loader=train_dataloader_encoded)

 Epoch  |  Train Loss  | Train Accuracy |  Elapsed 
---------------------------------------------------
   1    |   0.805952   |    0.720342    |   0.06   
   2    |   0.408035   |    0.867742    |   0.05   
   3    |   0.326965   |    0.893512    |   0.05   
   4    |   0.264668   |    0.917519    |   0.06   
   5    |   0.222546   |    0.928754    |   0.05   
   6    |   0.185481   |    0.946084    |   0.06   
   7    |   0.154569   |    0.953381    |   0.06   
   8    |   0.132851   |    0.960930    |   0.05   
   9    |   0.110153   |    0.968696    |   0.05   
  10    |   0.094203   |    0.973347    |   0.05   
  11    |   0.082572   |    0.979250    |   0.06   
  12    |   0.067419   |    0.983340    |   0.05   
  13    |   0.067806   |    0.981887    |   0.05   
  14    |   0.058978   |    0.985769    |   0.05   
  15    |   0.051345   |    0.987425    |   0.06   
  16    |   0.048748   |    0.987169    |   0.06   
  17    |   0.041176   |    0.989342    |   0.05   
  18    |   

In [67]:
predictions = valid(model, valid_dataloader_encoded) 

 Valid Loss  | Valid Accuracy |  Elapsed 
-----------------------------------------
  0.434957   |    0.891247    |   0.01   


### CNN Train & Valid with reviewText(topic+reviewText) and encoded_sub_topic

In [70]:
train(total_epoch=20, model=model, train_loader=train_dataloader)

 Epoch  |  Train Loss  | Train Accuracy |  Elapsed 
---------------------------------------------------
   1    |   0.905358   |    0.651170    |   0.06   
   2    |   0.343762   |    0.873280    |   0.05   
   3    |   0.245454   |    0.912439    |   0.05   
   4    |   0.194280   |    0.933248    |   0.05   
   5    |   0.149448   |    0.949553    |   0.05   
   6    |   0.127224   |    0.959220    |   0.05   
   7    |   0.108021   |    0.965439    |   0.05   
   8    |   0.086340   |    0.975471    |   0.05   
   9    |   0.070642   |    0.979512    |   0.05   
  10    |   0.057822   |    0.985410    |   0.05   
  11    |   0.052452   |    0.986183    |   0.05   
  12    |   0.044450   |    0.989288    |   0.05   
  13    |   0.035475   |    0.991825    |   0.05   
  14    |   0.035988   |    0.992239    |   0.05   
  15    |   0.030670   |    0.993481    |   0.06   
  16    |   0.030972   |    0.992964    |   0.06   
  17    |   0.028047   |    0.993323    |   0.06   
  18    |   

In [71]:
predictions = valid(model, valid_dataloader) 

 Valid Loss  | Valid Accuracy |  Elapsed 
-----------------------------------------
 12.787843   |    0.011504    |   0.01   


In [41]:
path = "/content/drive/MyDrive/Masters_Thesis/Models/NewModel"
saveModel(model=model, path=path)

All files saved


In [27]:
saveCSVValidationResult(predictions,'/content/sample_data')

### Data Preprocessing & DataLoader

In [5]:
def tokenize_data(text, tokenizer, max_length):
    tokens = tokenizer(text)[:max_length]
    length = len(tokens)
    return {'tokens': tokens, 'length': length}

def numericalize_data(example, vocab):
    ids = [vocab[token] for token in example['tokens']]
    return {'ids': ids}

def collate(batch, pad_index,target_list):
    batch_ids = [torch.tensor(i['ids']) for i in batch]  # Convert ids list to tensor
    batch_ids = pad_sequence(batch_ids, padding_value=torch.tensor(pad_index), batch_first=True)
    batch_length = [torch.tensor(i['length']) for i in batch]  # Convert length list to tensor
    batch_length = torch.stack(batch_length)
    batch_label = [torch.tensor(i[target_list]) for i in batch]  # Convert reviewText list to tensor
    batch_label = torch.stack(batch_label)
    batch_text = [(i['reviewText']) for i in batch]
    batch = {'ids': batch_ids,
             'length': batch_length,
             target_list: batch_label,
             'text': batch_text}
    return batch

def get_data_loaders(train_dataframe, valid_dataframe,target_list, max_length=256,min_freq = 5, train_batch_size=16, learning_rate=2e-5):
    df_train = pd.read_csv(train_dataframe)
    df_test = pd.read_csv(valid_dataframe)
    df_train = pd.concat([df_train, df_train['reviewText'].apply(tokenize_data, tokenizer=tokenizer, max_length=max_length).apply(pd.Series)], axis=1)
    df_test = pd.concat([df_test, df_test['reviewText'].apply(tokenize_data, tokenizer=tokenizer, max_length=max_length).apply(pd.Series)], axis=1)
    special_tokens = ['<unk>', '<pad>']
    tokens = df_train['tokens'].tolist()
    vocab = torchtext.vocab.build_vocab_from_iterator(tokens, min_freq=min_freq, specials=special_tokens)
    unk_index = vocab['<unk>']
    pad_index = vocab['<pad>']
    vocab.set_default_index(unk_index)
    df_train = pd.concat([df_train, df_train.apply(numericalize_data, vocab=vocab, axis=1).apply(pd.Series)], axis=1)
    df_test = pd.concat([df_test, df_test.apply(numericalize_data, vocab=vocab, axis=1).apply(pd.Series)], axis=1)
    df_train = df_train[['ids', target_list, 'length','reviewText']]
    df_test = df_test[['ids', target_list, 'length','reviewText']]
    vocab_size = len(vocab)
    output_dim = df_train[target_list].nunique()
    train = df_train.apply(lambda row: {
        'ids': row['ids'],
        'length': row['length'],
        target_list: row[target_list],
        'reviewText': row['reviewText']
        }, axis=1).tolist()
    validation = df_test.apply(lambda row: {
        'ids': row['ids'],
        'length': row['length'],
        target_list: row[target_list],
        'reviewText': row['reviewText']
        }, axis=1).tolist()
    batch_size = 64
    collate_fn = functools.partial(collate, pad_index=pad_index, target_list=target_list)
    train_dataloader = DataLoader(train, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
    valid_dataloader = DataLoader(validation, batch_size=batch_size, collate_fn=collate_fn)

    return train_dataloader, valid_dataloader, vocab_size, pad_index, output_dim, vocab

### CNN Model

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, 
                 dropout_rate, pad_index, filter_sizes=[3, 4, 5], num_filters=[100, 100, 100]):

        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_index)
        
        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=embedding_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        self.fc = nn.Linear(sum(num_filters), output_dim)  # modify the input dimension of the linear layer
        self.dropout = nn.Dropout(dropout_rate)
        self.flatten = nn.Flatten()  # add a flatten layer
        
    def forward(self, ids, length):
        # Get embeddings from `input_ids`. Output shape: (b, max_len, embed_dim)
        x_embed = self.embedding(ids).float()

        # Permute `x_embed` to match input shape requirement of `nn.Conv1d`.
        # Output shape: (b, embed_dim, max_len)
        x_reshaped = x_embed.permute(0, 2, 1)

        # Apply CNN and ReLU. Output shape: (b, num_filters[i], L_out)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]
        
        # Concatenate x_pool_list to feed the fully connected layer.
        # Output shape: (b, sum(num_filters))
        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)
        
        x_fc = self.flatten(x_fc)  
        
        # Compute logits. Output shape: (b, n_classes)
        prediction = self.fc(self.dropout(x_fc))
        return prediction

In [17]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.zeros_(m.bias)
    elif isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if 'bias' in name:
                nn.init.zeros_(param)
            elif 'weight' in name:
                nn.init.orthogonal_(param)

def apply_weights(model,vocab):
  model.apply(initialize_weights)
  pretrained_embedding = vectors.get_vecs_by_tokens(vocab.get_itos())
  model.embedding.weight.data = pretrained_embedding
  return model

### Training & Validation

In [15]:
def train(total_epoch, model, train_loader):
    print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Train Accuracy':^12} | {'Elapsed':^9}")
    print("-"*51)
    for epoch in range(total_epoch):
      t0_epoch = time.time() 
      tr_loss = 0
      n_correct = 0
      nb_tr_steps = 0
      nb_tr_examples = 0
      model.train()
      for _,data in enumerate(train_loader, 0):
      # for batch in tqdm.tqdm(dataloader, desc='training...', file=sys.stdout):
        input_ids  = data['ids'].to(device)
        length = data['length']
        targets  = data[target_list].to(device)
        # print(input_ids)
        outputs = model(input_ids,length)

        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        n_correct += get_accuracy(outputs, targets).item()
        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

      time_elapsed = (time.time() - t0_epoch)/60
      epoch_loss = tr_loss/nb_tr_steps
      epoch_accu = n_correct/nb_tr_steps # np.mean(n_correct)
      print(f"{epoch + 1:^7} | {epoch_loss:^12.6f} | {epoch_accu:^14.6f} | {time_elapsed:^9.2f}")

def valid(model, testing_loader):
    print(f"{'Valid Loss':^12} | {'Valid Accuracy':^12} | {'Elapsed':^9}")
    print("-"*41)
    model.eval()
    t0_epoch = time.time() 
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    predictions = []
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
        #   input_ids, targets, text = tuple(t.to(device) for t in data)
          input_ids  = data['ids'].to(device)
          length = data['length']
          targets  = data[target_list].to(device)
          text = data['text']
          outputs = model(input_ids,length)
          

          loss = loss_function(outputs, targets)
          tr_loss += loss.item()
          n_correct += get_accuracy(outputs, targets).item()

          nb_tr_steps += 1
          nb_tr_examples+=targets.size(0)

          #Todo: get text
          for i in range(len(text)):
            predictions.append({
                'text': text[i],
                'predicted': switch_issue(outputs.argmax(dim=-1)[i].item()),
                'target': switch_issue(targets[i].item())
                })

    time_elapsed = (time.time() - t0_epoch)/60                
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = n_correct/nb_tr_steps # np.mean(n_correct)
    print(f"{epoch_loss:^12.6f} | {epoch_accu:^14.6f} | {time_elapsed:^9.2f}")
    return predictions

def switch_issue(issue_type):
    switcher = {
      4: 'Product Description Issue',
      3: 'Delivery and Return Issue',
      2: 'Design Issue',
      1: 'Quality Issue',
      0: 'Product Authenticity Issue'
      }
    return switcher.get(issue_type, "Invalid Issue Type")

def get_accuracy(prediction, label):
    batch_size, _ = prediction.shape
    predicted_classes = prediction.argmax(dim=-1)
    correct_predictions = predicted_classes.eq(label).sum()
    accuracy = correct_predictions / batch_size
    return accuracy


### Save Files

In [26]:
def saveCSVValidationResult(predictions, path, fileName="/back_only_01/result.csv"):
    path = path+"/"+fileName
    with open(path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Text', 'Predicted', 'Target'])
        for example in predictions:
            writer.writerow([example['text'], example['predicted'], example['target']])

In [24]:
def saveModel(model, path):
  MODEL_PATH = path+'/back_only_01/model.pth'
  torch.save(model.state_dict(), MODEL_PATH)  
  print('All files saved')

def loadModel(path):
  MODEL_PATH = path+'/model.pth'
  model.load_state_dict(torch.load(MODEL_PATH))
  return model, tokenizer